# Installation

In [ ]:
# Before you start, please make sure Docker is installed and running on your system.
# Download and run
# docker pull qdrant/qdrant

In [ ]:
# Then, run the service:
# docker run -p 6333:6333 -p 6334:6334 -v "$(pwd)/qdrant_storage:/qdrant/storage:z" qdrant/qdrant
"""Theo cấu hình mặc định, tất cả dữ liệu sẽ được lưu trữ trong thư mục ./qdrant_storage. 
Đây cũng sẽ là thư mục duy nhất mà cả Container và máy chủ đều có thể nhìn thấy."""

# Qdrant is now accessible:
# REST API: localhost:6333
# Web UI: localhost:6333/dashboard
# GRPC API: localhost:6334

# Initialize the client

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")
# or
# client = QdrantClient(host="localhost", port=6333)

# Theo mặc định, Qdrant bắt đầu mà không có mã hóa hoặc xác thực.
# Điều này có nghĩa là bất kỳ ai có quyền truy cập mạng vào máy của bạn đều có thể truy 
# cập vào phiên bản container Qdrant của bạn.
# Vui lòng đọc kỹ Bảo mật để biết chi tiết về cách bảo mật phiên bản của bạn.

# Create a collection

In [5]:
from qdrant_client.models import Distance
from qdrant_client.models import VectorParams

client.create_collection(
    collection_name="est_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT)
)
# Lưu trữ dữ liệu vector trong bộ sưu tập qdrant "test_collection"
# Collection này sử dụng số liệu khoảng cách vô hướng để so sánh vector

True

# Add vectors

In [4]:
# Hãy thêm một vài vector với một payload
# Payload là một dữ liệu khác mà bạn muốn liên kết đến vector
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ]
)
print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


# Run a query

In [5]:
# Hãy đặt một câu hỏi cơ bản - Trong các vectơ được lưu trữ của chúng ta, 
# vectơ nào giống nhất với vectơ truy vấn [0,2, 0,1, 0,9, 0,7]?
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=True,
    limit=3,
).points

print(search_result)

# Kết quả trả về xếp giảm dần dựa theo sự tương đồng

[ScoredPoint(id=4, version=0, score=1.362, payload={'city': 'New York'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=1.273, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=1.208, payload={'city': 'Moscow'}, vector=None, shard_key=None, order_value=None)]


# Add a filter

In [6]:
# Chúng ta có thể thu hẹp kết quả hơn nữa bằng cách lọc theo tải trọng. Hãy tìm kết quả gần nhất có chứa "London".
from qdrant_client.models import Filter
from qdrant_client.models import FieldCondition
from qdrant_client.models import MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(must=[FieldCondition(key="city", match=MatchValue(value="London"))]),
    with_payload=True,
    limit=3,
).points

print(search_result)

[ScoredPoint(id=2, version=0, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]


# Các phương pháp search vector

1. Vector search (Similarity search): Tìm kiếm theo theo độ chương đồng dựa vào khoảng cách
    - Cosine: đo độ tương đồng theo góc
    - Dot: tích vô hướng
    - Euclidean: khoảng cách L2

2. Filtered search (Tìm kiếm có điều kiện)

3. Hybrid search (kết hợp vector search và keyword search)

4. Grouped search (Tìm kiếm có nhóm)

5. Các loại phương pháp search khác